<a href="https://colab.research.google.com/github/MarcelaFerreiraR/brazilian-stock-analysis/blob/main/Brazilian_Stock_Market_Analysis_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Baixar as bibliotecas
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# URL do TradingView
url = "https://br.tradingview.com/markets/stocks-brazil/market-movers-active/"

# Ler a página (usando requests e BeautifulSoup)
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extrair tabelas (ajustar para encontrar a tabela correta, se necessário)
tables = pd.read_html(str(soup.find('table')))[0]  # Pegar a primeira tabela encontrada

# Selecionar colunas relevantes (Ticker e Setor, removendo Volume completamente)
dados = tables.iloc[:, [0, -2]].copy()  # Pegar colunas 0 (Ticker) e penúltima (Setor), removendo Volume
dados.columns = ['Ticker', 'Setor']  # Renomear colunas, excluindo Volume

# Função para extrair o ticker correto (4 letras + 1 número)
dados['Ticker'] = dados['Ticker'].str.extract(r'([A-Z]{4}[0-9])')

# Preencher dados vazios no Ticker com 'B3SA3'
dados['Ticker'] = dados['Ticker'].fillna('B3SA3')

# Lista de tickers limpos
tickers = dados['Ticker'].tolist()
tickers = [ticker + ".SA" for ticker in tickers]  # Adicionar ".SA" a cada ticker


In [ ]:
# Baixar os dados do Yahoo Finance usando yfinance
end_date = datetime.now().date()
start_date = datetime(2022, 1, 1).date()
dados_acoes = yf.download(tickers, start=start_date, end=end_date)

# Verificar o estado inicial de dados_acoes antes de qualquer manipulação
print("\nEstado inicial de dados_acoes (colunas e índice):")
print("Colunas:", dados_acoes.columns)
print("Índice:", dados_acoes.index)

In [ ]:
print(dados_acoes.info())

In [ ]:
print(dados_acoes.head())

In [ ]:
# Resetar o índice para tornar 'Date' uma coluna (mantendo 'Date' como coluna)
dados_acoes = dados_acoes.reset_index()

In [ ]:
print(dados_acoes.info())

In [ ]:
# Restaurar 'Date' como índice para facilitar a manipulação com stack
dados_acoes = dados_acoes.set_index(('Date', ''))

# Usar stack para transformar de wide para long, mantendo o MultiIndex nas colunas
dados_long = dados_acoes.stack(level='Ticker')

# Resetar o índice para ter todas as variáveis como colunas
dados_long = dados_long.reset_index()

In [ ]:
print(dados_long.info())

In [ ]:
print(dados_long.head())

In [ ]:
# Verificar o resultado
print("\nDataFrame após stack e reset_index (long format):")
print(dados_long.head())
print("Colunas após transformação:", dados_long.columns)

In [ ]:
dados_long.columns = [col[0] if isinstance(col, tuple) else col for col in dados_long.columns]

In [ ]:
dados_long = dados_long.rename(columns={"Date": "date"})
dados_long = dados_long.rename(columns={'Ticker': 'acao'})

In [ ]:
# Converter a coluna 'Date' para o formato Date
dados_long['date'] = pd.to_datetime(dados_long['date'])

# Criar colunas de período
dados_long['dia'] = dados_long['date'].dt.date  # Dia (data completa)
dados_long['semana'] = dados_long['date'].dt.strftime('%U/%Y')  # Semana (WW/YY)
dados_long['quinzenal'] = dados_long['date'].apply(lambda x: f"{x.day <= 15 and '01' or '16'}/{x.year}")  # Quinzena (01 ou 16/ano)
dados_long['mes'] = dados_long['date'].dt.strftime('%b')  # Mês (MM/YY)
dados_long['ano'] = dados_long['date'].dt.strftime('%Y')  # Ano

print(dados_long.head())

In [ ]:
# Verificar o resultado
print("\nDataFrame após stack e reset_index (long format):")
print(dados_long.head())
print("Colunas após transformação:", dados_long.columns)

In [ ]:
# Verificar novamente após o filtro para VALE3.SA
print("\nDataFrame após filtro para 'Close' e conversão para VALE3.SA:")
print(dados_long[dados_long['acao'] == 'VALE3.SA']['date'].head(10))
print("Número de dias únicos para VALE3.SA após filtro:", dados_long[dados_long['acao'] == 'VALE3.SA']['date'].nunique())
print("Gaps detectados em VALE3.SA após filtro:")
dates = dados_long[dados_long['acao'] == 'VALE3.SA']['date'].sort_values()
gaps = dates.diff().dt.days > 1
print(dates[gaps])

In [ ]:
# Ordenar dados_long por 'date' e 'acao' para garantir consecutividade
dados_long = dados_long.sort_values(['date', 'acao'])

# Verificar o número de dias por ticker antes de calcular os retornos
print("\nNúmero de dias por ticker em 'dados_long':")
for ticker in tickers:
    days_count = dados_long[dados_long['acao'] == ticker]['date'].nunique()
    print(f"{ticker}: {days_count} dias únicos")

In [ ]:
# Análise detalhada dos valores vazios na coluna Retorno_ticker (para depuração, mas não impacta o resultado final)
print("Quantidade de valores NaN em 'Retorno_ticker':", dados_long.isna().sum())
print("Porcentagem de valores NaN em 'Retorno_ticker':", (dados_long.isna().sum() / len(dados_long)) * 100, "%")

In [ ]:
print(dados_long.info())

In [ ]:
# Usar Close para cálculos de retorno
dados_long['retorno'] = dados_long.groupby('acao')['Close'].transform(lambda x: x.pct_change() * 100)

In [ ]:
print(dados_long.info())
print(dados_long.head())

In [ ]:
print(dados_long.retorno.tail())

In [ ]:
def analisar_pares(acao1, acao2, dados):
    """
    Função para análise de pares de ativos (Long & Short)

    Parâmetros:
    - acao1: Primeiro ativo para análise
    - acao2: Segundo ativo para análise
    - dados: DataFrame com colunas 'date', 'acao', 'retorno'
    """
    try:
        # Pivotear os dados para ter retornos por ação
        df = dados.pivot(index='date', columns='acao', values='retorno')

        # Verificar se as ações existem no DataFrame
        if acao1 not in df.columns or acao2 not in df.columns:
            raise ValueError(f"Erro: {acao1} ou {acao2} não encontradas nos dados!")

        # Remover valores NaN
        df_filtrado = df[[acao1, acao2]].dropna()

        # Correlação
        correlacao = df_filtrado[acao1].corr(df_filtrado[acao2])
        print(f"Correlação entre {acao1} e {acao2}: {correlacao:.4f}")

        # Beta
        X = sm.add_constant(df_filtrado[acao2])
        y = df_filtrado[acao1]
        model = sm.OLS(y, X).fit()
        beta = model.params[acao2]
        print(f"Beta de {acao1} em relação a {acao2}: {beta:.4f}")

        # Teste de Cointegração
        score, p_value, _ = coint(df_filtrado[acao1], df_filtrado[acao2])
        print(f"Teste de Cointegração p-value: {p_value:.4f}")

        # Plotar Spread
        spread = df_filtrado[acao1] - beta * df_filtrado[acao2]
        plt.figure(figsize=(12, 6))
        plt.plot(df_filtrado.index, spread, label="Spread")
        plt.axhline(spread.mean(), color='red', linestyle='--', label="Média do Spread")
        plt.title(f"Spread entre {acao1} e {acao2}")
        plt.xlabel("Data")
        plt.ylabel("Spread")
        plt.legend()
        plt.tight_layout()
        plt.show()

        # Retornar métricas adicionais
        return {
            'correlacao': correlacao,
            'beta': beta,
            'p_value_coint': p_value,
            'spread_mean': spread.mean(),
            'spread_std': spread.std()
        }

    except Exception as e:
        print(f"Erro na análise de pares: {e}")
        return None

In [ ]:
# Supondo que você tenha um DataFrame chamado 'dados_acoes'
resultado = analisar_pares('ANIM3.SA', 'ASAI3.SA', dados_long)
